In [75]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import nltk
from string import punctuation
from nltk.corpus import stopwords



LM_dict=pd.read_excel('LoughranMcDonald_MasterDictionary_2020.xlsx')
LM_dict.drop(columns= ['Seq_num', 'Word Count', 'Word Proportion','Average Proportion', 'Std Dev', 'Doc Count','Uncertainty', 'Litigious', 'Strong_Modal', 'Weak_Modal','Constraining', 'Complexity', 'Syllables', 'Source'],inplace=True)


LM_dict['Word']=LM_dict['Word'].str.lower()                           #making separate copy for positive and negative words
LM_pos=LM_dict.copy()
LM_neg=LM_dict.copy()

   
et=LM_pos[LM_pos['Positive']==0].index                                       #sorting positive values
LM_pos.drop(et,inplace=True)
LM_pos.drop(['Negative'],axis=1,inplace=True)
LM_pos=LM_pos.reset_index(drop=True)


rt=LM_neg[LM_neg['Negative']==0].index                                        #sorting negative values
LM_neg.drop(rt,inplace=True)
LM_neg.drop(['Positive'],axis=1,inplace=True)
LM_neg=LM_neg.reset_index(drop=True)


    
a = open('StopWords_Generic.txt', 'r')                                           #loading stopwords_generic.txt and removing garbage values
for i in a:
    stpds+=[i[:-1].lower()]    
    
    
unc_dict=pd.read_excel('uncertainty_dictionary.xlsx')                            #loading uncertainty_dictionary.xlsx and converting them to lower
unc_dict['Word']=unc_dict['Word'].str.lower()

con_dict=pd.read_excel('constraining_dictionary.xlsx')                           #loading constraining_dictionary.xlsx and converting them to lower
con_dict['Word']=con_dict['Word'].str.lower()



def pos_neg(t):                                                                  #creating a function to pre process the converted text file so that finding positive and negative can done easily that are not present in the provided set of stopwords
    all_words=[]
    for sent in nltk.sent_tokenize(t.lower()):
        words = nltk.word_tokenize(sent)        
        words = [word for word in words if word not in stpds] 
        words = [word for word in words if word not in punctuation]
        words = [word for word in words if not word.isnumeric()]
        all_words+=words
    all_words = ''.join([ch for ch in ' '.join(all_words) if (ch not in punctuation) and (not ch.isnumeric())])
    return all_words 


def texts(t):                                                                        #creating a function to pre process the converted text file so that sentence and words and easily found that are not present in the provided set of stopwords
    all_words=[]
    for sent in nltk.sent_tokenize(t.lower()):
        words = nltk.word_tokenize(sent)        
        words = [word for word in words if word not in stpds] 
        words = [word for word in words if word not in new_punctuation]
        words = [word for word in words if not word.isnumeric()]
        all_words+=words
    all_words = ''.join([ch for ch in ' '.join(all_words) if (ch not in new_punctuation) and (not ch.isnumeric())])
    return all_words


def syllables(word):                                                                    #creating a function to count no of syllables in a word
    count = 0
    vowels = 'aeiouy'
    if word[0] in vowels:
        count +=1
    for index in range(1,len(word)):
        if word[index] in vowels and word[index-1] not in vowels:
            count +=1
    if word.endswith('e'):
        count -= 1
    if word.endswith('le'):
        count+=1
    if count == 0:
        count +=1
    return count


def cons_repo(t):                                                                               #creating a function to pre process the text file so that constraining words in the whole report can be found as described.
    all_words=[]
    for sent in nltk.sent_tokenize(t.lower()):
        words = nltk.word_tokenize(sent)        
        words = [word for word in words if word not in punctuation]
        words = [word for word in words if not word.isnumeric()]
        all_words+=words
    all_words = ''.join([ch for ch in ' '.join(all_words) if (ch not in punctuation) and (not ch.isnumeric())])
    return all_words

for i in range(2,200):                                                                          #creating a list yu which contain the undesired patterns of " . " and space.
    p='.'*i
    yu+=[p]
yu.reverse()
for i in range(1,60):  
    p='.'+(' '*i)+'.'
    yu+=[p]
for i in range(2,300):     
    p=' '*i
    yu+=[p]




cik=pd.read_excel('cik_list.xlsx')                                                         #loading cik_list.xlsx and getting SECFNAME to get links.
for i in cik['SECFNAME']:
    links+=['https://www.sec.gov/Archives/' + i]
    
     
header={                                                                                                                                                            #header file to connect to site and get the data from links
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36",
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9"
}


last=pd.DataFrame(columns=['positive_score','negative_score','polarity_score','average_sentence_length','percentage_of_complex_words','fog_index','complex_word_count','word_count','uncertainty_score','constraining_score','positive_word_proportion','negative_word_proportion','uncertainty_word_proportion','constraining_word_proportion','constraining_words_whole_report'])



for i in range (0,len(links)):                                      #for loop to run all the links
    
    
    links=[]                                                         # empty variables that are used to store values in the program
    stpds=[]       
    yu=[]
    sas=[]
    wees=0
    report=''
    negative=0
    positive=0
    complex_wrd=0
    word_count=0
    constraining_score=0
    uncertainity_score=0
    cons_wrd_whl_repo=0
    new_punctuation=punctuation[:13]+punctuation[14:]
  
    
    r=requests.get(links[i],headers=header)                                               # requesting the links and creating a soup to store the data
    text=r.text
    soup=BeautifulSoup(text,'html.parser')

    
    for wui in soup.find_all('page'):                                                       # from the soup we rectified the desired data and removed any html,css,xml present.
        report+=wui.get_text()
    if not report:
        for wui in soup.find_all('text'):
            report+=wui.get_text()
        
        
        
    ter=pos_neg(report)                                                                     # we used split function to seperate the data from spaces and created a list named ty which contained all the words that will be processed to find no. of positive and negative words
    ty=ter.split()
    
    for g in ty:                                                                            # finding positive , negative words and polarity
        if g in LM_neg['Word'].tolist():
            negative+=1
        if g in LM_pos['Word'].tolist():
            positive+=1
    polarity=(positive-negative)/((positive+negative)+0.000001)                          
    
    
    wer=texts(report)                                                                       # using text function created above to sort the data accordingling that is then used to find no. of words and sentences.
    for u in yu:                                                                            # removing the patterns of " . " and spaces from data.
        wer=wer.replace(u,' ')
        
          
    sentences=wer.count(' . ')                                                               #counting no. of sentences
    if not sentences:
        sentences=wer.count('. ')
    if not sentences:
        sentences=wer.count('.')
           
        
    qw=wer.split()                                                                                                     
    for k in qw:                                                                                   
        if (len(k)>1) and (k[-1]!='.'):
            sas+=[k.lower()]                                                                           # creating a list sas that is later used as it contain all the rectified words
            wees+=1                                                                                    # finding no. of words
    
    avg_sentence_length=wees/sentences                                                       # calculating average sentence length
    
    for n in sas:                                                                            # finding complex words
        if syllables(n)>2:
            complex_wrd+=1
        
    percentage_clx_wrds=complex_wrd/wees                                                    # calculating percentage of complex words
    fog_index=0.4*(avg_sentence_length+percentage_clx_wrds)                                 # calculating fog index
    
    for m in sas:
        if m not in stopwords.words('english'):                                              # calculating word count
            word_count+=1
    
    for l in sas:                                                                            # finding uncertainity score
        if l in unc_dict['Word'].tolist():
            uncertainity_score+=1

    for n in sas:                                                                            # finding constraining score
        if n in con_dict['Word'].tolist():
            constraining_score+=1
            
    pos_word_proportion=positive/word_count                                                  # finding positive word proportion
    
    neg_word_proportion=negative/word_count                                                  # finding negative word proportion
    
    uncertainity_wrd_pro=uncertainity_score/word_count                                       # finding uncertainity word proportion
    
    constraining_wrd_pro=constraining_score/word_count                                       # finding constraining word proportion
    
    ret=cons_repo(report)                                                                    # finding constraining word in the whole report 
    ret=ret.split()
    for e in ret:
        if e in con_dict['Word'].tolist():
            cons_wrd_whl_repo+=1
    
    
    last=last.append({'positive_score':positive,'negative_score':negative,'polarity_score':polarity,'average_sentence_length':avg_sentence_length,'percentage_of_complex_words':percentage_clx_wrds,'fog_index':fog_index,'complex_word_count':complex_wrd,'word_count':word_count,'uncertainty_score':uncertainity_score,'constraining_score':constraining_score,'positive_word_proportion':pos_word_proportion,'negative_word_proportion':neg_word_proportion,'uncertainty_word_proportion':uncertainity_wrd_pro,'constraining_word_proportion':constraining_wrd_pro,'constraining_words_whole_report':cons_wrd_whl_repo },ignore_index=True)

                                                                                            # adding calculated values to the dataframa.
    
resultant=pd.concat([cik,last],axis=1)                                                      # concating cik_list.xlsx with the calculated dataframe sidewise as described.
resultant.to_excel("resultant.xlsx")                                                        # saving the created dataframe to the drive.
        
        
    





